# Document Pipeline

## Handles PDF, PPTX and Txt

In [1]:
import os
import json
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
from pptx import Presentation

class DocumentReader:
    """A class for reading and extracting text from various document formats."""

    def __init__(self, file_path):
        """
        Initialize the DocumentReader with a file path.

        Args:
            file_path (str): Path to the document file

        Raises:
            FileNotFoundError: If the specified file does not exist
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"The file {file_path} does not exist.")
            
        self.file_path = file_path
        self.file_extension = os.path.splitext(file_path)[1].lower()

    def extract_text(self):
        """
        Extract text content from the document based on its file type.

        Returns:
            str: Extracted text content from the document

        Raises:
            ValueError: If the file type is not supported
        """
        handlers = {
            '.txt': self._extract_text_from_txt,
            '.pdf': self._extract_text_from_pdf,
            '.pptx': self._extract_text_from_pptx
        }

        handler = handlers.get(self.file_extension)
        if handler:
            return handler()
        else:
            raise ValueError(f"Unsupported file extension: {self.file_extension}")

    def _extract_text_from_txt(self):
        """
        Extract text from a plain text file.

        Returns:
            str: Content of the text file
        """
        with open(self.file_path, 'r', encoding='utf-8') as file:
            return file.read()

    def _extract_text_from_pdf(self):
        """
        Extract text from a PDF file, including text from images using OCR.

        Returns:
            str: Combined text content from all PDF pages
        """
        doc = fitz.open(self.file_path)
        text_contents = []

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text("text")

            if text.strip():
                text_contents.append(text)
            else:
                # If no text is found, try OCR
                text_contents.append(self._extract_text_from_pdf_image(page))

        return '\n'.join(text_contents)

    def _extract_text_from_pdf_image(self, page):
        """
        Extract text from an image-based PDF page using OCR.

        Args:
            page: PyMuPDF Page object

        Returns:
            str: Extracted text from the page image
        """
        zoom = 2.0  # Higher zoom factor for better OCR accuracy
        matrix = fitz.Matrix(zoom, zoom)
        pixmap = page.get_pixmap(matrix=matrix)
        
        image = Image.frombytes(
            "RGB", 
            [pixmap.width, pixmap.height], 
            pixmap.samples
        )
        
        return pytesseract.image_to_string(image)

    def _extract_text_from_pptx(self):
        """
        Extract text from a PowerPoint presentation.

        Returns:
            str: Combined text content from all slides
        """
        presentation = Presentation(self.file_path)
        text_contents = []

        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text_contents.append(shape.text)

        return '\n'.join(text_contents)


def parse_json(json_string):
    """
    Parse a JSON string into a Python object.

    Args:
        json_string (str): JSON string to parse, optionally with markdown code block markers

    Returns:
        dict/list: Parsed JSON object

    Raises:
        json.JSONDecodeError: If the JSON string is invalid
    """
    cleaned_string = json_string.replace('```json', '').replace('```', '').strip()
    
    return json.loads(cleaned_string)

In [3]:
file_path = ['docs/sample.pptx', 'docs/soil.pdf', 'docs/sample.txt']
extracted_files = []
for file in file_path:
    reader = DocumentReader(file)
    text = reader.extract_text()
    extracted_files.append(text)
extracted_files

['Various Image Segmentation Techniques:\nDifferent types of Image Segmentation Techniques\nThresholding technique segmentation\nHistogram based segmentation\nRegion based segmentation\nEdge based segmentation\nClustering based segmentation\nMorphological Transforms and\nTexture based segmentation approaches\nThresholding technique segmentation\nSegmentation algorithms based on thresholding approach are suitable for images where there is distinct difference between object and background.\nMain Goal: divide an image into two distinct regions (object and background) directly based on intensity values and their properties\nTypes: Global, Variable, Multiple\n\nOriginal coins image\n2)Histogram based segmentation\nHistogram of an image is a plot between intensity levels. \nDeep valleys are used to separate different peaks of histogram. \nHistogram peaks are tall, narrow, symmetric.\n3) Region based Segmentation\nThe region-based segmentation methods segments the image into various regions h

# Youtube Pipeline

# Imports

In [12]:
import os
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from typing import List, Tuple
import os
import shutil
import yt_dlp
from moviepy.editor import AudioFileClip
import re
import requests

# Downloading Youtube Audio

In [13]:
def setup_output_directory(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def download_youtube_video_and_audio(video_url, output_dir='Saved_Media'):
    setup_output_directory(output_dir)

    # Commented as the video download is not required
    # video_options = {
    #     'format': 'best',
    #     'outtmpl': os.path.join(output_dir, 'video.%(ext)s'),
    # }

    audio_options = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, 'audio.%(ext)s'),
    }

    # for download_options in [video_options, audio_options]:

    with yt_dlp.YoutubeDL(audio_options) as youtube_downloader:
        youtube_downloader.download([video_url])

    print("Video and audio download completed!")

def convert_audio_to_mp3(input_audio_path, output_mp3_path):
    audio = AudioFileClip(input_audio_path)
    audio.write_audiofile(output_mp3_path)
    audio.close()
    print("Audio conversion to MP3 completed!")

def process_youtube_video(video_url):
    output_dir = 'Saved_Media'
    
    download_youtube_video_and_audio(video_url, output_dir)
    
    input_audio_path = os.path.join(output_dir, 'audio.webm')
    output_mp3_path = os.path.join(output_dir, 'audio.mp3')
    
    convert_audio_to_mp3(input_audio_path, output_mp3_path)

def clean_captions(raw_captions):
    lines = raw_captions.decode('utf-8').split('\n')
    cleaned_captions = []
    timestamp_pattern = re.compile(r'\d{2}:\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}:\d{2}\.\d{3}')
    
    for line in lines:
        if line and not line.startswith('WEBVTT') and not timestamp_pattern.match(line):
            cleaned_line = line.strip()
            if cleaned_line:
                cleaned_captions.append(cleaned_line)
    
    return ' '.join(cleaned_captions)

def extract_subtitles(video_url):
    subtitle_options = {
        'writesubtitles': True,
        'subtitleslangs': ['en'],
        'subtitlesformat': 'vtt',
        'skip_download': True,
    }

    with yt_dlp.YoutubeDL(subtitle_options) as youtube_downloader:
        info_dict = youtube_downloader.extract_info(video_url, download=False)
        subtitles = info_dict.get('requested_subtitles', None)
        
        if subtitles:
            print(f"Subtitles available for {video_url}")
            for lang, subtitle_info in subtitles.items():
                subtitle_url = subtitle_info.get('url')
                if subtitle_url:
                    response = requests.get(subtitle_url)
                    if response.status_code == 200:
                        raw_subtitles = response.content
                        return clean_captions(raw_subtitles)

In [14]:
class WhisperTranscriber:
    def __init__(self, model_name: str = "openai/whisper-tiny"):
        """
        Initialize the WhisperTranscriber with a specified model.

        Args:
            model_name (str): The name of the Whisper model to use.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.model.eval()

    @staticmethod
    def load_audio(file_path: str, target_sampling_rate: int = 16000) -> Tuple[np.ndarray, int]:
        """
        Load and resample audio to 16kHz if necessary using librosa.

        Args:
            file_path (str): Path to the audio file.
            target_sampling_rate (int): Target sampling rate (default: 16000).

        Returns:
            Tuple[np.ndarray, int]: Resampled audio array and sampling rate.
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Audio file not found: {file_path}")

        try:
            audio, sr = librosa.load(file_path, sr=target_sampling_rate, mono=True)
            return audio, sr
        except Exception as e:
            raise RuntimeError(f"Error loading audio file: {e}. Make sure the file format is supported by librosa.")

    @staticmethod
    def chunk_audio(audio: np.ndarray, chunk_length: int = 30, sampling_rate: int = 16000) -> List[np.ndarray]:
        """
        Split audio into chunks for long transcriptions.

        Args:
            audio (np.ndarray): The audio array.
            chunk_length (int): Length of each chunk in seconds (default: 30).
            sampling_rate (int): Sampling rate of the audio (default: 16000).

        Returns:
            List[np.ndarray]: List of audio chunks.
        """
        chunk_size = chunk_length * sampling_rate
        return [audio[i:i+chunk_size] for i in range(0, len(audio), chunk_size)]

    def transcribe_chunk(self, chunk: np.ndarray, sampling_rate: int = 16000) -> str:
        """
        Transcribe a single audio chunk.

        Args:
            chunk (np.ndarray): Audio chunk to transcribe.
            sampling_rate (int): Sampling rate of the audio (default: 16000).

        Returns:
            str: Transcribed text.
        """
        input_features = self.processor(chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        input_features = input_features.to(self.device)

        with torch.no_grad():
            predicted_ids = self.model.generate(input_features)

        return self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    def transcribe(self, file_path: str, chunk_length: int = 30) -> str:
        """
        Transcribe long audio by chunking.

        Args:
            file_path (str): Path to the audio file.
            chunk_length (int): Length of each chunk in seconds (default: 30).

        Returns:
            str: Full transcription of the audio.
        """
        audio, sampling_rate = self.load_audio(file_path)
        chunks = self.chunk_audio(audio, chunk_length, sampling_rate)
        
        transcriptions = [self.transcribe_chunk(chunk, sampling_rate) for chunk in chunks]
        return " ".join(transcriptions)

# Testing out the pipeline

In [15]:
def get_youtube_transcript(youtube_url: str) -> str:
    try:
        existing_subtitles = extract_subtitles(youtube_url)
        if existing_subtitles:
            print("Subtitles found ... Extracted Subtitles from YouTube Video...")
            return existing_subtitles
        else:
            print("Getting YouTube Video's Audio...")
            process_youtube_video(youtube_url)
            audio_file = r"Saved_Media\audio.mp3"
            if not os.path.exists(audio_file):
                raise FileNotFoundError(f"Audio file not found: {audio_file}")
            
            print("Transcribing Audio...")
            transcriber = WhisperTranscriber()
            transcribed_text = transcriber.transcribe(audio_file)
            return transcribed_text
    except Exception as e:
        print(f"Error: {e}")
        return ""

youtube_url = "https://www.youtube.com/watch?v=3NgmslFf33I"
transcript = get_youtube_transcript(youtube_url)
print(transcript)

[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I: Downloading ios player API JSON
[youtube] 3NgmslFf33I: Downloading web creator player API JSON
[youtube] 3NgmslFf33I: Downloading m3u8 information
Getting YouTube Video's Audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I: Downloading ios player API JSON
[youtube] 3NgmslFf33I: Downloading web creator player API JSON
[youtube] 3NgmslFf33I: Downloading m3u8 information
[info] 3NgmslFf33I: Downloading 1 format(s): 251-5
[download] Destination: Saved_Media\audio.webm
[download] 100% of    7.66MiB in 00:00:04 at 1.66MiB/s   
Video and audio download completed!
MoviePy - Writing audio in Saved_Media\audio.mp3


MoviePy - Done.
Audio conversion to MP3 completed!
Transcribing Audio...
Using device: cuda


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Hi, welcome to another video. So Google has launched their new Gemini 2.0 flash model. Yes, the Gemini 2.0 flash model is now available for users to use. It is initially marked as experimental, but this is majorly the final form of this model. And this is the first frontier model in their two points.  0 lineup. To be honest, it's the first next gen model that we are seeing since OpenAI is still stuck on GPT4 and clawed on 3.5 so on it. But this is the first new frontier model by Google. They say that is twice as fast as pro while achieving stronger performance. The major part about this model is that it comes with native multi-modal outputs and native tool use.  Basically, Gemini 2.0 flash can now generate and edit images in itself without hitting any other end point. It can also generate audio outputs as well within itself, which is just amazing. It's like GPT40, but it can also generate images, which is pretty amazing. They say that Gemini 2.0 flash features native text to speech au

# Medium Articles

In [4]:
import requests
from bs4 import BeautifulSoup
import json

def get_medium_article_content(url):
    try:
        # Send request to the URL
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find article content
        article_content = ""
        
        # Get title
        title = soup.find('h1')
        if title:
            article_content += f"Title: {title.get_text()}\n\n"
        
        # Get main article content
        article = soup.find('article')
        if article:
            # Get all paragraphs
            paragraphs = article.find_all('p')
            for p in paragraphs:
                article_content += f"{p.get_text()}\n\n"
        
        return article_content.strip()
        
    except requests.exceptions.RequestException as e:
        return f"Error fetching the article: {str(e)}"
    except Exception as e:
        return f"Error processing the article: {str(e)}"

In [5]:
get_medium_article_content('https://medium.com/@pdx.lucasm/canvas-with-react-js-32e133c05258')

'Title: Canvas with React.js\n\nLucas Miranda\n\nFollow\n\n--\n\n12\n\nListen\n\nShare\n\nIn this article, we will see how to create a Canvas React component and a custom hook for extracting its logic, so we can just draw inside it like we usually draw in a regular canvas html element.\n\nThis article is based on Corey’s article “Animating a Canvas with React Hooks”. Any other sources and related contents are linked throughout this article.\n\nI am assuming that you already know canvas, but if you don’t know yet, I recommend this tutorial from MDN to you.\n\nIn order to see what we are doing, let’s create a new react app with create-react-app (feel free to skip this step if you are already familiar with React and create-react-app). You can start a new project by running npx create-react-app example or yarn create react-app example if you prefer yarn. If you open the project folder (example) in your code editor, you must get something like this:\n\nWe don’t need all these files, so we w

# Merged Code for all total Pipeline

In [13]:
from typing import Dict,Any
import os
import json
import fitz
import pytesseract
from PIL import Image
from pptx import Presentation
import requests
from bs4 import BeautifulSoup
from typing import Union, Dict, Any
from transcribe import WhisperTranscriber
from utils import *
def get_youtube_transcript(youtube_url: str) -> str:
    try:
        existing_subtitles = extract_subtitles(youtube_url)
        if existing_subtitles:
            print("Subtitles found ... Extracted Subtitles from YouTube Video...")
            return existing_subtitles
        else:
            print("Getting YouTube Video's Audio...")
            process_youtube_video(youtube_url)
            audio_file = r"Saved_Media\audio.mp3"
            if not os.path.exists(audio_file):
                raise FileNotFoundError(f"Audio file not found: {audio_file}")
            
            print("Transcribing Audio...")
            transcriber = WhisperTranscriber()
            transcribed_text = transcriber.transcribe(audio_file)
            return transcribed_text
    except Exception as e:
        print(f"Error: {e}")
        return ""


class DocumentExtractor:
    """A unified class for extracting content from various document types including web content."""
    
    def __init__(self):
        """Initialize the document extractor with necessary components."""
        self.youtube_transcriber = WhisperTranscriber()
        
    def extract_content(self, source: str) -> Dict[str, Any]:
        """
        Extract content from various document types including web links.
        
        Args:
            source (str): File path or URL to the content
            
        Returns:
            Dict[str, Any]: Dictionary containing:
                - 'content': The extracted text/content
                - 'metadata': Additional information about the extraction
                - 'type': The type of content that was processed
        """
        try:
            # Check if source is a URL
            if source.startswith(('http://', 'https://')):
                if 'youtube.com' in source or 'youtu.be' in source:
                    return self._extract_youtube_content(source)
                elif 'medium.com' in source:
                    return self._extract_medium_content(source)
                else:
                    raise ValueError("Unsupported URL type")
            
            # Handle local files
            if not os.path.exists(source):
                raise FileNotFoundError(f"The file {source} does not exist.")
                
            file_extension = os.path.splitext(source)[1].lower()
            
            handlers = {
                '.txt': self._extract_text_file,
                '.pdf': self._extract_pdf,
                '.pptx': self._extract_pptx
            }
            
            handler = handlers.get(file_extension)
            if not handler:
                raise ValueError(f"Unsupported file type: {file_extension}")
                
            return handler(source)
            
        except Exception as e:
            return {
                'content': '',
                'metadata': {'error': str(e)},
                'type': 'error'
            }
    
    def _extract_text_file(self, file_path: str) -> Dict[str, Any]:
        """Extract content from text files."""
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return {
            'content': content,
            'metadata': {'file_size': os.path.getsize(file_path)},
            'type': 'text'
        }
    
    def _extract_pdf(self, file_path: str) -> Dict[str, Any]:
        """Extract content from PDF files with OCR support."""
        doc = fitz.open(file_path)
        text_contents = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text("text")
            
            if not text.strip():
                # Try OCR if no text is found
                zoom = 2.0
                matrix = fitz.Matrix(zoom, zoom)
                pixmap = page.get_pixmap(matrix=matrix)
                image = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
                text = pytesseract.image_to_string(image)
                
            text_contents.append(text)
            
        return {
            'content': '\n'.join(text_contents),
            'metadata': {'pages': len(doc)},
            'type': 'pdf'
        }
    
    def _extract_pptx(self, file_path: str) -> Dict[str, Any]:
        """Extract content from PowerPoint presentations."""
        presentation = Presentation(file_path)
        text_contents = []
        
        for slide in presentation.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    slide_text.append(shape.text)
            text_contents.append(' '.join(slide_text))
            
        return {
            'content': '\n'.join(text_contents),
            'metadata': {'slides': len(presentation.slides)},
            'type': 'pptx'
        }
    
    def _extract_youtube_content(self, url: str) -> Dict[str, Any]:
        """Extract transcript from YouTube videos."""
        try:
            transcript = get_youtube_transcript(url)
            return {
                'content': transcript,
                'metadata': {'url': url},
                'type': 'youtube'
            }
        except Exception as e:
            raise RuntimeError(f"Failed to extract YouTube content: {str(e)}")
    
    def _extract_medium_content(self, url: str) -> Dict[str, Any]:
        """Extract content from Medium articles."""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract title
            title = soup.find('h1').get_text() if soup.find('h1') else ''
            
            # Extract article content
            article_content = []
            content_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
            
            for element in content_elements:
                text = element.get_text().strip()
                if text:
                    article_content.append(text)
            
            return {
                'content': '\n'.join(article_content),
                'metadata': {
                    'title': title,
                    'url': url
                },
                'type': 'medium'
            }
            
        except Exception as e:
            raise RuntimeError(f"Failed to extract Medium content: {str(e)}")


In [14]:
extractor = DocumentExtractor()
    
sources = [
        'docs/sample.txt',
        'docs/soil.pdf',
        'docs/sample.pptx',
        'https://medium.com/@pdx.lucasm/canvas-with-react-js-32e133c05258',
        'https://www.youtube.com/watch?v=3NgmslFf33I'
    ]

for source in sources:
    try:
        result = extractor.extract_content(source)
        print(f"\nProcessed {source}:")
        print(f"Type: {result['type']}")
        print(f"Content length: {len(result['content'])} characters")
        print(f"Metadata: {result['metadata']}")
    except Exception as e:
        print(f"Error processing {source}: {str(e)}")

Using device: cuda

Processed docs/sample.txt:
Type: text
Content length: 990 characters
Metadata: {'file_size': 1011}

Processed docs/soil.pdf:
Type: pdf
Content length: 15926 characters
Metadata: {'pages': 38}

Processed docs/sample.pptx:
Type: pptx
Content length: 3566 characters
Metadata: {'slides': 14}

Processed https://medium.com/@pdx.lucasm/canvas-with-react-js-32e133c05258:
Type: medium
Content length: 9166 characters
Metadata: {'title': 'Canvas with React.js', 'url': 'https://medium.com/@pdx.lucasm/canvas-with-react-js-32e133c05258'}
[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I: Downloading ios player API JSON
[youtube] 3NgmslFf33I: Downloading web creator player API JSON
[youtube] 3NgmslFf33I: Downloading m3u8 information
Getting YouTube Video's Audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I

MoviePy - Done.
Audio conversion to MP3 completed!
Transcribing Audio...
Using device: cuda


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Processed https://www.youtube.com/watch?v=3NgmslFf33I:
Type: youtube
Content length: 6953 characters
Metadata: {'url': 'https://www.youtube.com/watch?v=3NgmslFf33I'}


In [36]:
result

{'content': " Hi, welcome to another video. So Google has launched their new Gemini 2.0 flash model. Yes, the Gemini 2.0 flash model is now available for users to use. It is initially marked as experimental, but this is majorly the final form of this model. And this is the first frontier model in their two points.  0 lineup. To be honest, it's the first next gen model that we are seeing since OpenAI is still stuck on GPT4 and clawed on 3.5 so on it. But this is the first new frontier model by Google. They say that is twice as fast as pro while achieving stronger performance. The major part about this model is that it comes with native multi-modal outputs and native tool use.  Basically, Gemini 2.0 flash can now generate and edit images in itself without hitting any other end point. It can also generate audio outputs as well within itself, which is just amazing. It's like GPT40, but it can also generate images, which is pretty amazing. They say that Gemini 2.0 flash features native text

In [3]:
from pipeline import DocumentExtractor
extractor = DocumentExtractor()

Using device: cuda


In [4]:
try:
    result = extractor.extract_content('https://www.youtube.com/watch?v=3NgmslFf33I')
    print(result)
except Exception as e:
    print(f"Error: {e}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I: Downloading ios player API JSON
[youtube] 3NgmslFf33I: Downloading web creator player API JSON
[youtube] 3NgmslFf33I: Downloading m3u8 information
Getting YouTube Video's Audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=3NgmslFf33I
[youtube] 3NgmslFf33I: Downloading webpage
[youtube] 3NgmslFf33I: Downloading ios player API JSON
[youtube] 3NgmslFf33I: Downloading web creator player API JSON
[youtube] 3NgmslFf33I: Downloading m3u8 information
[info] 3NgmslFf33I: Downloading 1 format(s): 251-5
[download] Destination: Saved_Media\audio.webm
[download] 100% of    7.66MiB in 00:00:05 at 1.31MiB/s   
Video and audio download completed!
MoviePy - Writing audio in Saved_Media\audio.mp3


MoviePy - Done.
Audio conversion to MP3 completed!
Transcribing Audio...
Using device: cuda
{'content': " Hi, welcome to another video. So Google has launched their new Gemini 2.0 flash model. Yes, the Gemini 2.0 flash model is now available for users to use. It is initially marked as experimental, but this is majorly the final form of this model. And this is the first frontier model in their two points.  0 lineup. To be honest, it's the first next gen model that we are seeing since OpenAI is still stuck on GPT4 and clawed on 3.5 so on it. But this is the first new frontier model by Google. They say that is twice as fast as pro while achieving stronger performance. The major part about this model is that it comes with native multi-modal outputs and native tool use.  Basically, Gemini 2.0 flash can now generate and edit images in itself without hitting any other end point. It can also generate audio outputs as well within itself, which is just amazing. It's like GPT40, but it can also g

In [ ]:
try:
    result = extractor.extract_content('https://www.youtube.com/watch?v=3NgmslFf33I')
    print(result)
except Exception as e:
    print(f"Error: {e}")